#### 임상정보 csv        
           [case2] age, Overall.Stage, gender  -> 성능이 더 높은 case2 채택
    train: 'clinical_info_case2_train.csv'   
    test:  'clinical_info_case2_test.csv'   
#### 종양내부 csv    
    train: 'patient_features_intra_train.csv'  
    test: 'patient_features_intra_test.csv'
#### 종양외부 csv  
    train: 'patient_features_peri_train.csv'  
    test: 'patient_features_peri_test.csv'
#### 종양내외부 csv
    train: 'patient_features_combine_train.csv'
    test: 'patient_features_combine_test.csv'
--------------------
### 임상정보 + 종양내부(diff) |  종양내부 + 종양외부(ok)  |  임상정보 + 종양내외부 

In [1]:
import pandas as pd
import numpy as np

import random
random.seed(42)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier

## 1. 데이터 로드

- clinical_info_case2_train / clinical_info_case2_test - 임상정보 case2

In [12]:
clinical_info_case2_train=pd.read_csv("clinical_info_case2_train.csv")
clinical_info_case2_train['PatientID']=clinical_info_case2_train['PatientID'].apply(lambda x: str(x).zfill(3))
clinical_info_case2_train=clinical_info_case2_train.set_index('PatientID')
#clinical_info_case2_train['age'] = np.where(clinical_info_case2_train['age']<=65, 0, 1)
clinical_info_case2_train

,age,Overall_Stage,gender,two_year_survival
PatientID,,,,
250,65,3,1,0
131,71,1,1,1
200,66,2,1,1
114,79,3,0,0
109,77,2,1,1
...,...,...,...,...
146,67,3,1,1
222,78,4,1,0
073,60,4,1,1


In [13]:
clinical_info_case2_test=pd.read_csv("clinical_info_case2_test.csv")
clinical_info_case2_test['PatientID']=clinical_info_case2_test['PatientID'].apply(lambda x: str(x).zfill(3))
clinical_info_case2_test=clinical_info_case2_test.set_index('PatientID')
#clinical_info_case2_test['age'] = np.where(clinical_info_case2_test['age']<=65, 0, 1)
clinical_info_case2_test

,age,Overall_Stage,gender,two_year_survival
PatientID,,,,
005,80,4,1,1
006,73,3,1,1
030,62,4,1,1
042,72,4,1,1
045,72,2,0,0
047,46,3,1,0
063,72,2,0,0
096,85,4,1,1
123,77,4,0,1


- patient_features_intra_train / patient_features_intra_test - 종양내부

In [85]:
patient_features_intra_train=pd.read_csv("patient_features_intra_train.csv")
patient_features_intra_train['PatientID']=patient_features_intra_train['PatientID'].apply(lambda x: str(x).zfill(3))
patient_features_intra_train=patient_features_intra_train.set_index('PatientID')
patient_features_intra_train

,intra_firstorder_10Percentile,intra_firstorder_RobustMeanAbsoluteDeviation,intra_firstorder_MeanAbsoluteDeviation,intra_firstorder_InterquartileRange,intra_glcm_JointEntropy,intra_firstorder_Entropy,intra_gldm_DependenceNonUniformityNormalized,intra_glcm_SumEntropy,intra_firstorder_Uniformity,intra_gldm_DependenceEntropy,...,intra_glszm_GrayLevelNonUniformity,intra_glrlm_LongRunHighGrayLevelEmphasis,intra_glszm_LargeAreaHighGrayLevelEmphasis,intra_glrlm_LongRunEmphasis,intra_glszm_LargeAreaEmphasis,intra_glrlm_LongRunLowGrayLevelEmphasis,intra_glszm_LargeAreaLowGrayLevelEmphasis,intra_ngtdm_Busyness,intra_shape2D_Elongation,recurrence
PatientID,,,,,,,,,,,,,,,,,,,,,
250,173.8,12.934749,21.421470,26.00,2.208903,1.639827,0.532299,1.966279,0.441840,2.299817,...,32.028169,2669.408451,2669.408451,77.316901,77.316901,2.500732,2.500732,0.601256,0.689679,0
250,188.0,8.072490,15.841121,16.00,2.117787,1.494682,0.522875,1.845018,0.464949,2.227720,...,33.442748,1521.511450,1521.511450,63.732824,63.732824,3.104177,3.104177,1.118233,0.588182,0
250,175.2,13.934228,20.324333,35.00,2.312891,1.683575,0.497577,2.008172,0.394718,2.501142,...,22.936842,1023.610526,1023.610526,44.705263,44.705263,2.361582,2.361582,0.795578,0.561224,0
131,170.0,9.938241,22.964176,16.25,2.269777,1.654881,0.573174,2.008540,0.477403,2.435155,...,57.140625,5189.570312,5189.570312,110.601562,110.601562,3.782793,3.782793,0.899623,0.557147,1
131,180.0,9.768869,19.358870,18.00,2.030152,1.494781,0.601407,1.816971,0.517042,2.108752,...,57.719101,6198.322097,6198.322097,176.104869,176.104869,5.660643,5.660643,1.022230,0.771689,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,101.0,34.990773,44.894697,84.00,4.155986,2.917565,0.470471,3.637555,0.151346,4.007719,...,142.737452,1077.547297,1077.547297,23.706564,23.706564,1.537288,1.537288,3.106743,0.890817,1
284,150.0,19.067193,26.179600,44.00,3.230650,2.268701,0.483114,2.902607,0.242226,3.307392,...,104.556840,980.019268,980.019268,27.578035,27.578035,1.014181,1.014181,2.431955,0.783703,1
151,221.0,2.467243,10.546802,5.00,1.108511,0.816777,0.751707,0.980880,0.770886,1.261634,...,68.041667,29242.333333,29242.333333,599.534722,599.534722,12.541908,12.541908,0.694496,0.803645,1


In [86]:
patient_features_intra_test=pd.read_csv("patient_features_intra_test.csv")
patient_features_intra_test['PatientID']=patient_features_intra_test['PatientID'].apply(lambda x: str(x).zfill(3))
patient_features_intra_test=patient_features_intra_test.set_index('PatientID')
patient_features_intra_test

,intra_firstorder_10Percentile,intra_firstorder_RobustMeanAbsoluteDeviation,intra_firstorder_MeanAbsoluteDeviation,intra_firstorder_InterquartileRange,intra_glcm_JointEntropy,intra_firstorder_Entropy,intra_gldm_DependenceNonUniformityNormalized,intra_glcm_SumEntropy,intra_firstorder_Uniformity,intra_gldm_DependenceEntropy,...,intra_glszm_GrayLevelNonUniformity,intra_glrlm_LongRunHighGrayLevelEmphasis,intra_glszm_LargeAreaHighGrayLevelEmphasis,intra_glrlm_LongRunEmphasis,intra_glszm_LargeAreaEmphasis,intra_glrlm_LongRunLowGrayLevelEmphasis,intra_glszm_LargeAreaLowGrayLevelEmphasis,intra_ngtdm_Busyness,intra_shape2D_Elongation,recurrence
PatientID,,,,,,,,,,,,,,,,,,,,,
005,216.0,6.113426,11.502419,14.00,1.692196,1.223053,0.641084,1.536160,0.586309,1.929795,...,174.132137,6040.047308,6040.047308,171.727569,171.727569,5.032538,5.032538,3.033041,0.321134,1
006,99.0,38.762735,50.060614,90.00,3.550693,2.580647,0.505011,3.121539,0.253428,3.480407,...,187.939775,4321.093316,4321.093316,73.380543,73.380543,2.990132,2.990132,3.292700,0.572172,1
030,186.0,9.671757,22.150999,19.50,2.554708,1.852613,0.555030,2.292216,0.411292,2.678433,...,142.357143,4445.131868,4445.131868,91.918956,91.918956,2.417148,2.417148,2.254514,0.437281,1
042,226.0,3.329783,9.458135,7.00,0.976124,0.761863,0.804361,0.899854,0.793335,1.071575,...,117.047538,52140.120543,52140.120543,1450.748727,1450.748727,41.023647,41.023647,2.014534,0.834098,1
045,207.0,4.696030,10.467795,8.00,1.495275,1.123687,0.635903,1.377701,0.623508,1.778120,...,109.709046,3738.173594,3738.173594,152.420538,152.420538,6.430153,6.430153,2.110561,0.290531,0
047,125.0,34.445385,40.125214,87.00,3.496558,2.515687,0.478368,3.160634,0.198562,3.528714,...,117.958032,1053.269175,1053.269175,31.037627,31.037627,2.163028,2.163028,3.130580,0.497265,0
063,227.0,2.417465,7.698029,4.00,0.777071,0.581961,0.781251,0.708592,0.836422,0.996161,...,38.095652,27859.991304,27859.991304,570.391304,570.391304,11.988826,11.988826,0.324322,0.757860,0
096,157.0,13.468700,27.929567,18.00,2.020580,1.545337,0.676286,1.825781,0.546365,2.062192,...,193.107931,31131.557645,31131.557645,492.924775,492.924775,9.391714,9.391714,1.915978,0.582015,1
123,185.0,5.682038,17.830396,6.00,1.450733,1.124756,0.707664,1.309963,0.667582,1.567746,...,102.449735,20798.358025,20798.358025,428.185185,428.185185,9.171644,9.171644,1.140882,0.549730,1


- patient_features_peri_train / patient_features_peri_test - 종양외부(15mm)

In [103]:
patient_features_peri_train=pd.read_csv("patient_features_peri_train.csv")
patient_features_peri_train['PatientID']=patient_features_peri_train['PatientID'].apply(lambda x: str(x).zfill(3))
patient_features_peri_train=patient_features_peri_train.set_index('PatientID')
patient_features_peri_train

,peri_glcm_Idm,peri_glcm_Id,peri_gldm_LargeDependenceEmphasis,peri_firstorder_Median,peri_glrlm_RunPercentage,peri_glszm_ZonePercentage,peri_gldm_DependenceNonUniformityNormalized,peri_glcm_DifferenceEntropy,peri_gldm_SmallDependenceEmphasis,peri_glcm_InverseVariance,...,peri_ngtdm_Busyness,peri_firstorder_10Percentile,peri_glcm_Imc2,peri_ngtdm_Contrast,peri_glrlm_ShortRunLowGrayLevelEmphasis,peri_glszm_SmallAreaLowGrayLevelEmphasis,peri_firstorder_Kurtosis,peri_firstorder_Minimum,peri_shape2D_PerimeterSurfaceRatio,recurrence
PatientID,,,,,,,,,,,,,,,,,,,,,
250,0.938651,0.941210,7.754086,94.0,0.126899,0.126899,0.627535,0.607547,0.153411,0.088255,...,9.028602,0.0,0.967094,0.037978,0.007753,0.007753,1.669493,0.0,0.079338,0
250,0.940181,0.942418,7.781214,91.0,0.124082,0.124082,0.633201,0.589047,0.152311,0.087005,...,10.774463,0.0,0.968075,0.042450,0.018016,0.018016,1.442181,0.0,0.079139,0
250,0.944002,0.946309,7.835643,90.0,0.118951,0.118951,0.645541,0.558595,0.151839,0.080158,...,13.948310,0.0,0.962778,0.054873,0.011241,0.011241,1.264349,0.0,0.079738,0
131,0.915553,0.920560,7.463265,89.0,0.156816,0.156816,0.572805,0.793644,0.164273,0.103133,...,7.121987,0.0,0.965189,0.025245,0.022966,0.022966,3.495972,0.0,0.079697,1
131,0.898002,0.902295,7.077157,94.0,0.196374,0.196374,0.518225,0.874966,0.178156,0.149264,...,6.337776,71.0,0.972808,0.025770,0.021132,0.021132,3.683667,0.0,0.079072,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,0.927607,0.930196,7.544443,94.0,0.148337,0.148337,0.586902,0.684579,0.160814,0.110551,...,7.574558,0.0,0.976015,0.018157,0.014306,0.014306,2.690847,0.0,0.079988,1
284,0.915112,0.917572,7.300108,99.0,0.173895,0.173895,0.546930,0.751729,0.171351,0.137332,...,6.269245,0.0,0.971852,0.016641,0.012782,0.012782,4.381179,0.0,0.079640,1
151,0.883131,0.888447,6.808222,81.0,0.223896,0.223896,0.492589,0.958037,0.187722,0.171770,...,11.188188,63.0,0.963373,0.020169,0.014408,0.014408,5.517176,0.0,0.079923,1


In [104]:
patient_features_peri_test=pd.read_csv("patient_features_peri_test.csv")
patient_features_peri_test['PatientID']=patient_features_peri_test['PatientID'].apply(lambda x: str(x).zfill(3))
patient_features_peri_test=patient_features_peri_test.set_index('PatientID')
patient_features_peri_test

,peri_glcm_Idm,peri_glcm_Id,peri_gldm_LargeDependenceEmphasis,peri_firstorder_Median,peri_glrlm_RunPercentage,peri_glszm_ZonePercentage,peri_gldm_DependenceNonUniformityNormalized,peri_glcm_DifferenceEntropy,peri_gldm_SmallDependenceEmphasis,peri_glcm_InverseVariance,...,peri_ngtdm_Busyness,peri_firstorder_10Percentile,peri_glcm_Imc2,peri_ngtdm_Contrast,peri_glrlm_ShortRunLowGrayLevelEmphasis,peri_glszm_SmallAreaLowGrayLevelEmphasis,peri_firstorder_Kurtosis,peri_firstorder_Minimum,peri_shape2D_PerimeterSurfaceRatio,recurrence
PatientID,,,,,,,,,,,,,,,,,,,,,
005,0.891431,0.896858,6.899566,224.0,0.214920,0.214920,0.499292,0.899523,0.185713,0.157694,...,4.962536,90.0,0.986254,0.042833,0.017823,0.017823,2.565295,55.0,0.086429,1
006,0.895185,0.899103,6.949021,98.0,0.209308,0.209308,0.505133,0.853588,0.182118,0.167590,...,17.359034,72.0,0.984313,0.045937,0.050035,0.050035,2.106897,55.0,0.088564,1
030,0.899448,0.905118,7.084129,189.0,0.196810,0.196810,0.517280,0.836721,0.181738,0.142319,...,4.177470,85.0,0.990171,0.047185,0.019415,0.019415,1.333240,58.0,0.086748,1
042,0.903754,0.908109,7.127535,188.0,0.192216,0.192216,0.522798,0.801304,0.179690,0.147626,...,6.997736,104.0,0.989973,0.043750,0.030556,0.030556,1.428819,77.0,0.090538,1
045,0.929375,0.930445,7.423431,211.0,0.160163,0.160163,0.566485,0.622747,0.163259,0.130230,...,5.608043,118.0,0.988660,0.021297,0.016084,0.016084,1.514219,93.0,0.085794,0
047,0.900014,0.902786,6.996968,113.0,0.205107,0.205107,0.508504,0.797897,0.182765,0.171613,...,24.693532,90.0,0.977217,0.042262,0.056166,0.056166,2.471468,76.0,0.080675,0
063,0.917952,0.921009,7.355007,218.0,0.168130,0.168130,0.555280,0.719568,0.168907,0.132184,...,3.746565,97.0,0.981134,0.042117,0.032247,0.032247,1.652708,83.0,0.079247,0
096,0.881492,0.886289,6.739252,114.0,0.232188,0.232188,0.484112,0.913806,0.194286,0.187346,...,12.283456,82.0,0.982671,0.044196,0.025969,0.025969,2.192480,56.0,0.097744,1
123,0.924678,0.927262,7.418466,109.0,0.161532,0.161532,0.565366,0.683199,0.166304,0.122737,...,5.581354,77.0,0.986768,0.039754,0.025101,0.025101,1.238245,55.0,0.083867,1


- patient_features_combine_train / patient_features_combine_test - 종양내외부

In [3]:
patient_features_combine_train=pd.read_csv("patient_features_combine_train.csv")
patient_features_combine_train['PatientID']=patient_features_combine_train['PatientID'].apply(lambda x: str(x).zfill(3))
patient_features_combine_train=patient_features_combine_train.set_index('PatientID')
patient_features_combine_train

,peri_glcm_Idm,peri_glcm_Id,peri_gldm_LargeDependenceEmphasis,peri_firstorder_Median,peri_glrlm_RunPercentage,peri_glszm_ZonePercentage,peri_gldm_DependenceNonUniformityNormalized,peri_glcm_DifferenceEntropy,peri_gldm_SmallDependenceEmphasis,peri_glcm_InverseVariance,...,intra_glrlm_LongRunEmphasis,intra_glszm_LargeAreaEmphasis,peri_firstorder_Kurtosis,peri_firstorder_Minimum,intra_glrlm_LongRunLowGrayLevelEmphasis,intra_glszm_LargeAreaLowGrayLevelEmphasis,intra_ngtdm_Busyness,peri_shape2D_PerimeterSurfaceRatio,intra_shape2D_Elongation,recurrence
PatientID,,,,,,,,,,,,,,,,,,,,,
250,0.938651,0.941210,7.754086,94.0,0.126899,0.126899,0.627535,0.607547,0.153411,0.088255,...,77.316901,77.316901,1.669493,0.0,2.500732,2.500732,0.601256,0.079338,0.689679,0
250,0.940181,0.942418,7.781214,91.0,0.124082,0.124082,0.633201,0.589047,0.152311,0.087005,...,63.732824,63.732824,1.442181,0.0,3.104177,3.104177,1.118233,0.079139,0.588182,0
250,0.944002,0.946309,7.835643,90.0,0.118951,0.118951,0.645541,0.558595,0.151839,0.080158,...,44.705263,44.705263,1.264349,0.0,2.361582,2.361582,0.795578,0.079738,0.561224,0
131,0.915553,0.920560,7.463265,89.0,0.156816,0.156816,0.572805,0.793644,0.164273,0.103133,...,110.601562,110.601562,3.495972,0.0,3.782793,3.782793,0.899623,0.079697,0.557147,1
131,0.898002,0.902295,7.077157,94.0,0.196374,0.196374,0.518225,0.874966,0.178156,0.149264,...,176.104869,176.104869,3.683667,0.0,5.660643,5.660643,1.022230,0.079072,0.771689,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,0.927607,0.930196,7.544443,94.0,0.148337,0.148337,0.586902,0.684579,0.160814,0.110551,...,23.706564,23.706564,2.690847,0.0,1.537288,1.537288,3.106743,0.079988,0.890817,1
284,0.915112,0.917572,7.300108,99.0,0.173895,0.173895,0.546930,0.751729,0.171351,0.137332,...,27.578035,27.578035,4.381179,0.0,1.014181,1.014181,2.431955,0.079640,0.783703,1
151,0.883131,0.888447,6.808222,81.0,0.223896,0.223896,0.492589,0.958037,0.187722,0.171770,...,599.534722,599.534722,5.517176,0.0,12.541908,12.541908,0.694496,0.079923,0.803645,1


In [4]:
patient_features_combine_test=pd.read_csv("patient_features_combine_test.csv")
patient_features_combine_test['PatientID']=patient_features_combine_test['PatientID'].apply(lambda x: str(x).zfill(3))
patient_features_combine_test=patient_features_combine_test.set_index('PatientID')
patient_features_combine_test

,peri_glcm_Idm,peri_glcm_Id,peri_gldm_LargeDependenceEmphasis,peri_firstorder_Median,peri_glrlm_RunPercentage,peri_glszm_ZonePercentage,peri_gldm_DependenceNonUniformityNormalized,peri_glcm_DifferenceEntropy,peri_gldm_SmallDependenceEmphasis,peri_glcm_InverseVariance,...,intra_glrlm_LongRunEmphasis,intra_glszm_LargeAreaEmphasis,peri_firstorder_Kurtosis,peri_firstorder_Minimum,intra_glrlm_LongRunLowGrayLevelEmphasis,intra_glszm_LargeAreaLowGrayLevelEmphasis,intra_ngtdm_Busyness,peri_shape2D_PerimeterSurfaceRatio,intra_shape2D_Elongation,recurrence
PatientID,,,,,,,,,,,,,,,,,,,,,
005,0.891431,0.896858,6.899566,224.0,0.214920,0.214920,0.499292,0.899523,0.185713,0.157694,...,171.727569,171.727569,2.565295,55.0,5.032538,5.032538,3.033041,0.086429,0.321134,1
006,0.895185,0.899103,6.949021,98.0,0.209308,0.209308,0.505133,0.853588,0.182118,0.167590,...,73.380543,73.380543,2.106897,55.0,2.990132,2.990132,3.292700,0.088564,0.572172,1
030,0.899448,0.905118,7.084129,189.0,0.196810,0.196810,0.517280,0.836721,0.181738,0.142319,...,91.918956,91.918956,1.333240,58.0,2.417148,2.417148,2.254514,0.086748,0.437281,1
042,0.903754,0.908109,7.127535,188.0,0.192216,0.192216,0.522798,0.801304,0.179690,0.147626,...,1450.748727,1450.748727,1.428819,77.0,41.023647,41.023647,2.014534,0.090538,0.834098,1
045,0.929375,0.930445,7.423431,211.0,0.160163,0.160163,0.566485,0.622747,0.163259,0.130230,...,152.420538,152.420538,1.514219,93.0,6.430153,6.430153,2.110561,0.085794,0.290531,0
047,0.900014,0.902786,6.996968,113.0,0.205107,0.205107,0.508504,0.797897,0.182765,0.171613,...,31.037627,31.037627,2.471468,76.0,2.163028,2.163028,3.130580,0.080675,0.497265,0
063,0.917952,0.921009,7.355007,218.0,0.168130,0.168130,0.555280,0.719568,0.168907,0.132184,...,570.391304,570.391304,1.652708,83.0,11.988826,11.988826,0.324322,0.079247,0.757860,0
096,0.881492,0.886289,6.739252,114.0,0.232188,0.232188,0.484112,0.913806,0.194286,0.187346,...,492.924775,492.924775,2.192480,56.0,9.391714,9.391714,1.915978,0.097744,0.582015,1
123,0.924678,0.927262,7.418466,109.0,0.161532,0.161532,0.565366,0.683199,0.166304,0.122737,...,428.185185,428.185185,1.238245,55.0,9.171644,9.171644,1.140882,0.083867,0.549730,1


## 2. classification[분류모델 학습] & evaluation[성능평가]

### 2-1. 임상정보(case2) + 종양내부 통합

In [112]:
# LDA

solver = ['lsqr', 'eigen','svd']
shrinkage = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for i in range(20,81,20):
    print("------------- reliefF ",i,"-------------------------------------")
    for a in solver:
        for b in shrinkage:
            train=patient_features_intra_train.drop('recurrence',axis=1).iloc[:,:i]
            test=patient_features_intra_test.drop('recurrence',axis=1).iloc[:,:i]

            total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
            total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

            X_train=total1.drop('two_year_survival',axis=1).values
            X_test=total2.drop('two_year_survival',axis=1).values
            y_train=total1['two_year_survival'].values
            y_test=total2['two_year_survival'].values
    
            if a=='svd': # solver='svd'면 shrinkage 파라미더 지정 불가 이므로 
                lda=LinearDiscriminantAnalysis(solver=a)
            else:
                lda=LinearDiscriminantAnalysis(solver=a, shrinkage=b)
            clf = make_pipeline(StandardScaler(), lda)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            
            # 성능평가
            accuracy = metrics.accuracy_score(y_test, y_pred)
            sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
            specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
            ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
            npv = metrics.precision_score(y_test, y_pred, pos_label=0)
            if accuracy>0.7:
                print("solv:",a,",shrink:",b,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("------------- reliefF102-------------------------------------")
for a in solver:
    for b in shrinkage:
        train=patient_features_intra_train.drop('recurrence',axis=1)
        test=patient_features_intra_test.drop('recurrence',axis=1)

        total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
        total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

        X_train=total1.drop('two_year_survival',axis=1).values
        X_test=total2.drop('two_year_survival',axis=1).values
        y_train=total1['two_year_survival'].values
        y_test=total2['two_year_survival'].values

        if a=='svd': # solver='svd'면 shrinkage 파라미더 지정 불가 이므로 
            lda=LinearDiscriminantAnalysis(solver=a)
        else:
            lda=LinearDiscriminantAnalysis(solver=a, shrinkage=b)
        clf = make_pipeline(StandardScaler(),lda)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        # 성능평가
        accuracy = metrics.accuracy_score(y_test, y_pred)
        sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
        specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
        ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
        npv = metrics.precision_score(y_test, y_pred, pos_label=0)
        if accuracy>0.7:
            print("solv:",a,",shrink:",b,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

------------- reliefF  20 -------------------------------------
------------- reliefF  40 -------------------------------------
------------- reliefF  60 -------------------------------------
------------- reliefF  80 -------------------------------------
solv: lsqr ,shrink: 0.3 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
solv: lsqr ,shrink: 0.4 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
solv: lsqr ,shrink: 0.5 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
solv: lsqr ,shrink: 0.6 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
solv: eigen ,shrink: 0.3 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
solv: eigen ,shrink: 0.4 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
solv: eigen ,shrink: 0.5 / 0

In [113]:
# SVM

C_set = [0.1,1, 10, 50, 100]
kernel_set = ["rbf"]
gamma_set = [0.001, 0.01, 0.1,0.5, 1]

for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in C_set:
        for b in kernel_set:
            for c in gamma_set:
                train=patient_features_intra_train.drop('recurrence',axis=1).iloc[:,:i]
                test=patient_features_intra_test.drop('recurrence',axis=1).iloc[:,:i]

                total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
                total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

                X_train=total1.drop('two_year_survival',axis=1).values
                X_test=total2.drop('two_year_survival',axis=1).values
                y_train=total1['two_year_survival'].values
                y_test=total2['two_year_survival'].values

                clf = make_pipeline(StandardScaler(), SVC(C=a,kernel=b,gamma=c,degree=d,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                # 성능평가
                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>0.65:
                    print("c:",a,",k:",b,",g:",c,", d:",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("------------- reliefF102-------------------------------------")
for a in C_set:
    for b in kernel_set:
        for c in gamma_set:
            train=patient_features_intra_train.drop('recurrence',axis=1)
            test=patient_features_intra_test.drop('recurrence',axis=1)

            total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
            total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

            X_train=total1.drop('two_year_survival',axis=1).values
            X_test=total2.drop('two_year_survival',axis=1).values
            y_train=total1['two_year_survival'].values
            y_test=total2['two_year_survival'].values

            clf = make_pipeline(StandardScaler(), SVC(C=a,kernel=b,gamma=c,degree=d,random_state=42))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            # 성능평가
            accuracy = metrics.accuracy_score(y_test, y_pred)
            sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
            specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
            ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
            npv = metrics.precision_score(y_test, y_pred, pos_label=0)
            if accuracy>0.65:
                print("c:",a,",k:",b,",g:",c,", d:",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

-----------------------reliefF 20 ---------------------
-----------------------reliefF 40 ---------------------
-----------------------reliefF 60 ---------------------
-----------------------reliefF 80 ---------------------
------------- reliefF102-------------------------------------


In [107]:
# RF

n_estimators = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200]
max_depth = [None,2,4,6,8,10,12,14,16]
max_features_set = ["sqrt", "log2", "auto", None]

for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in n_estimators:
        for b in max_depth:
            for c in max_features_set:
                train=patient_features_intra_train.drop('recurrence',axis=1).iloc[:,:i]
                test=patient_features_intra_test.drop('recurrence',axis=1).iloc[:,:i]

                total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
                total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

                X_train=total1.drop('two_year_survival',axis=1).values
                X_test=total2.drop('two_year_survival',axis=1).values
                y_train=total1['two_year_survival'].values
                y_test=total2['two_year_survival'].values

                clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=a,max_depth=b,max_features=c,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                # 성능평가
                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>0.7:
                    print("n_e:",a,",m_d:",b,",m_f",c,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("------------- reliefF102-------------------------------------")
for a in n_estimators:
    for b in max_depth:
        for c in max_features_set:
            train=patient_features_intra_train.drop('recurrence',axis=1)
            test=patient_features_intra_test.drop('recurrence',axis=1)

            total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
            total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

            X_train=total1.drop('two_year_survival',axis=1).values
            X_test=total2.drop('two_year_survival',axis=1).values
            y_train=total1['two_year_survival'].values
            y_test=total2['two_year_survival'].values

            clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=a,max_depth=b,max_features=c,random_state=42))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            # 성능평가
            accuracy = metrics.accuracy_score(y_test, y_pred)
            sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
            specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
            ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
            npv = metrics.precision_score(y_test, y_pred, pos_label=0)
            if accuracy>0.7:
                print("n_e:",a,",m_d:",b,",m_f",c,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

-----------------------reliefF 20 ---------------------
n_e: 20 ,m_d: None ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: None ,m_f log2 / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: None ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 10 ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 10 ,m_f log2 / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 10 ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 12 ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 12 ,m_f log2 / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 12 ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0

n_e: 100 ,m_d: 12 ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 100 ,m_d: 12 ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 100 ,m_d: 14 ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 100 ,m_d: 14 ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 100 ,m_d: 16 ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 100 ,m_d: 16 ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 120 ,m_d: None ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 120 ,m_d: None ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 120 ,m_d: 2 ,m_f None / 0.75 ,  1.0 ,  0.2857142857142857 ,  0.7222222222222222 ,  1.0
n_e: 120 ,m_d: 8 ,m_f sqrt / 0.7 ,  0.84

In [108]:
# GB

n_estimators = [1,10,50, 100]
learning_rate = [0.1,0.3,0.5,0.6, 0.7,0.8,0.9]
subsample = [0.6,0.7,0.8,0.9,1.0]
max_depth = range(3,10,2)
    
for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in n_estimators:
        for b in learning_rate:
            for c in subsample:
                for d in max_depth:
                    train=patient_features_intra_train.drop('recurrence',axis=1).iloc[:,:i]
                    test=patient_features_intra_test.drop('recurrence',axis=1).iloc[:,:i]

                    total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
                    total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

                    X_train=total1.drop('two_year_survival',axis=1).values
                    X_test=total2.drop('two_year_survival',axis=1).values
                    y_train=total1['two_year_survival'].values
                    y_test=total2['two_year_survival'].values

                    clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=a,learning_rate=b, subsample=c,max_depth=d,random_state=42))
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    # 성능평가
                    accuracy = metrics.accuracy_score(y_test, y_pred)
                    sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                    specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                    ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                    npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                    if accuracy>=0.7 and sensitivity>0.5 and specificity>0.5:
                        print("n_e:",a,", l_r:",b,",s",c,",m_d",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("------------- reliefF102-------------------------------------")
for a in n_estimators:
    for b in learning_rate:
        for c in subsample:
            for d in max_depth:
                train=patient_features_intra_train.drop('recurrence',axis=1)
                test=patient_features_intra_test.drop('recurrence',axis=1)

                total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
                total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

                X_train=total1.drop('two_year_survival',axis=1).values
                X_test=total2.drop('two_year_survival',axis=1).values
                y_train=total1['two_year_survival'].values
                y_test=total2['two_year_survival'].values

                clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=a,learning_rate=b, subsample=c,max_depth=d,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                # 성능평가
                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>=0.7 and sensitivity>0.5 and specificity>0.5:
                    print("n_e:",a,", l_r:",b,",s",c,",m_d",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)


-----------------------reliefF 20 ---------------------
n_e: 50 , l_r: 0.8 ,s 0.9 ,m_d 3 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
-----------------------reliefF 40 ---------------------
-----------------------reliefF 60 ---------------------
n_e: 50 , l_r: 0.6 ,s 0.6 ,m_d 3 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
n_e: 100 , l_r: 0.6 ,s 0.6 ,m_d 7 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
-----------------------reliefF 80 ---------------------
n_e: 10 , l_r: 0.5 ,s 0.7 ,m_d 7 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
------------- reliefF102-------------------------------------
n_e: 10 , l_r: 0.6 ,s 0.6 ,m_d 5 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
n_e: 10 , l_r: 0.7 ,s 0.6 ,m_d 5 / 0.75 ,  0.6923076923076923 ,  0.85714285714

### 2-2. 종양내부 + 종양외부 통합

In [5]:
#LDA

solver = ['lsqr', 'eigen']
shrinkage = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    
for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in solver:
        for b in shrinkage:
            X_train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:i]
            X_test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:i]
            y_train=patient_features_combine_train['recurrence'].values
            y_test=patient_features_combine_test['recurrence'].values

            clf = make_pipeline(StandardScaler(), LinearDiscriminantAnalysis(solver=a, shrinkage=b))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            accuracy = metrics.accuracy_score(y_test, y_pred)
            sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
            specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
            ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
            npv = metrics.precision_score(y_test, y_pred, pos_label=0)
            if accuracy>0.7:
                print("solv:",a,",shrink:",b,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("-----------------------reliefF106---------------------")
for a in solver:
    for b in shrinkage:
        X_train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:102]
        X_test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:102]
        y_train=patient_features_combine_train['recurrence'].values
        y_test=patient_features_combine_test['recurrence'].values

        clf = make_pipeline(StandardScaler(), LinearDiscriminantAnalysis(solver=a, shrinkage=b))
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        accuracy = metrics.accuracy_score(y_test, y_pred)
        sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
        specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
        ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
        npv = metrics.precision_score(y_test, y_pred, pos_label=0)
        if accuracy>0.7:
            print("solv:",a,",shrink:",b,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

-----------------------reliefF 20 ---------------------
solv: lsqr ,shrink: 0.8 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
solv: lsqr ,shrink: 0.9 / 0.75 ,  0.9230769230769231 ,  0.42857142857142855 ,  0.75 ,  0.75
solv: eigen ,shrink: 0.8 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
solv: eigen ,shrink: 0.9 / 0.75 ,  0.9230769230769231 ,  0.42857142857142855 ,  0.75 ,  0.75
-----------------------reliefF 40 ---------------------
-----------------------reliefF 60 ---------------------
-----------------------reliefF 80 ---------------------
-----------------------reliefF106---------------------


In [7]:
#SVM

C_set = [0.1,1, 10, 50, 100]
kernel_set = ["rbf"]
gamma_set = [0.001, 0.01, 0.1,0.5, 1]

for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in C_set:
        for b in kernel_set:
            for c in gamma_set:
                X_train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:i]
                X_test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:i]
                y_train=patient_features_combine_train['recurrence'].values
                y_test=patient_features_combine_test['recurrence'].values

                clf = make_pipeline(StandardScaler(), SVC(C=a,kernel=b,gamma=c,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>0.7:
                    print("c:",a,",k:",b,",g",c,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("-----------------------reliefF106---------------------")
for a in C_set:
        for b in kernel_set:
            for c in gamma_set:
                X_train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:102]
                X_test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:102]
                y_train=patient_features_combine_train['recurrence'].values
                y_test=patient_features_combine_test['recurrence'].values
            
                clf = make_pipeline(StandardScaler(), SVC(C=a,kernel=b,gamma=c,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>0.7:
                    print("c:",a,",k:",b,",g",c,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

-----------------------reliefF 20 ---------------------
-----------------------reliefF 40 ---------------------
c: 50 ,k: rbf ,g 0.001 / 0.8 ,  1.0 ,  0.42857142857142855 ,  0.7647058823529411 ,  1.0
c: 50 ,k: rbf ,g 0.01 / 0.75 ,  1.0 ,  0.2857142857142857 ,  0.7222222222222222 ,  1.0
c: 100 ,k: rbf ,g 0.01 / 0.75 ,  1.0 ,  0.2857142857142857 ,  0.7222222222222222 ,  1.0
-----------------------reliefF 60 ---------------------
c: 10 ,k: rbf ,g 0.01 / 0.8 ,  0.6923076923076923 ,  1.0 ,  1.0 ,  0.6363636363636364
c: 50 ,k: rbf ,g 0.001 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
c: 100 ,k: rbf ,g 0.001 / 0.75 ,  0.8461538461538461 ,  0.5714285714285714 ,  0.7857142857142857 ,  0.6666666666666666
-----------------------reliefF 80 ---------------------
-----------------------reliefF106---------------------


In [8]:
#RF
n_estimators = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200]
max_depth = [None,2,4,6,8,10,12,14,16]
max_features_set = ["sqrt", "log2", "auto", None]

for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in n_estimators:
        for b in max_depth:
            for c in max_features_set:
                X_train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:i]
                X_test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:i]
                y_train=patient_features_combine_train['recurrence'].values
                y_test=patient_features_combine_test['recurrence'].values

                clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=a,max_depth=b,max_features=c,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>=0.8:
                    print("n_e:",a,",m_d:",b,",m_f",c,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("-----------------------reliefF106---------------------")
for a in n_estimators:
        for b in max_depth:
            for c in max_features_set:
                X_train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:102]
                X_test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:102]
                y_train=patient_features_combine_train['recurrence'].values
                y_test=patient_features_combine_test['recurrence'].values

                clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=a,max_depth=b,max_features=c,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>=0.8:
                    print("n_e:",a,",m_d:",b,",m_f",c,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

-----------------------reliefF 20 ---------------------
n_e: 60 ,m_d: 4 ,m_f sqrt / 0.8 ,  0.9230769230769231 ,  0.5714285714285714 ,  0.8 ,  0.8
n_e: 60 ,m_d: 4 ,m_f log2 / 0.8 ,  0.9230769230769231 ,  0.5714285714285714 ,  0.8 ,  0.8
n_e: 60 ,m_d: 4 ,m_f auto / 0.8 ,  0.9230769230769231 ,  0.5714285714285714 ,  0.8 ,  0.8
n_e: 80 ,m_d: 4 ,m_f sqrt / 0.8 ,  0.9230769230769231 ,  0.5714285714285714 ,  0.8 ,  0.8
n_e: 80 ,m_d: 4 ,m_f log2 / 0.8 ,  0.9230769230769231 ,  0.5714285714285714 ,  0.8 ,  0.8
n_e: 80 ,m_d: 4 ,m_f auto / 0.8 ,  0.9230769230769231 ,  0.5714285714285714 ,  0.8 ,  0.8
-----------------------reliefF 40 ---------------------
-----------------------reliefF 60 ---------------------
n_e: 100 ,m_d: 8 ,m_f sqrt / 0.8 ,  1.0 ,  0.42857142857142855 ,  0.7647058823529411 ,  1.0
n_e: 100 ,m_d: 8 ,m_f auto / 0.8 ,  1.0 ,  0.42857142857142855 ,  0.7647058823529411 ,  1.0
-----------------------reliefF 80 ---------------------
-----------------------reliefF106-------------------

In [10]:
#GB

n_estimators = [1,10,50, 100]
learning_rate = [0.1,0.3,0.5,0.6, 0.7,0.8,0.9]
subsample = [0.6,0.7,0.8,0.9,1.0]
max_depth = range(3,10,2)
    
for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in n_estimators:
        for b in learning_rate:
            for c in subsample:
                for d in max_depth:
                    X_train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:i]
                    X_test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:i]
                    y_train=patient_features_combine_train['recurrence'].values
                    y_test=patient_features_combine_test['recurrence'].values

                    clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=a,learning_rate=b, subsample=c,max_depth=d,random_state=42))
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    accuracy = metrics.accuracy_score(y_test, y_pred)
                    sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                    specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                    ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                    npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                    if accuracy>0.8:
                        print("n_e:",a,", l_r:",b,",s",c,",m_d",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("-----------------------reliefF106---------------------")
for a in n_estimators:
    for b in learning_rate:
        for c in subsample:
            for d in max_depth:
                X_train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:102]
                X_test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:102]
                y_train=patient_features_combine_train['recurrence'].values
                y_test=patient_features_combine_test['recurrence'].values

                clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=a,learning_rate=b, subsample=c,max_depth=d,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>0.8:
                    print("n_e:",a,", l_r:",b,",s",c,",m_d",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)


-----------------------reliefF 20 ---------------------
-----------------------reliefF 40 ---------------------
-----------------------reliefF 60 ---------------------
n_e: 50 , l_r: 0.6 ,s 0.7 ,m_d 9 / 0.85 ,  0.9230769230769231 ,  0.7142857142857143 ,  0.8571428571428571 ,  0.8333333333333334
n_e: 50 , l_r: 0.7 ,s 0.6 ,m_d 5 / 0.85 ,  0.9230769230769231 ,  0.7142857142857143 ,  0.8571428571428571 ,  0.8333333333333334
n_e: 50 , l_r: 0.8 ,s 0.7 ,m_d 7 / 0.85 ,  1.0 ,  0.5714285714285714 ,  0.8125 ,  1.0
n_e: 50 , l_r: 0.8 ,s 0.7 ,m_d 9 / 0.9 ,  1.0 ,  0.7142857142857143 ,  0.8666666666666667 ,  1.0
n_e: 100 , l_r: 0.6 ,s 0.6 ,m_d 7 / 0.85 ,  1.0 ,  0.5714285714285714 ,  0.8125 ,  1.0
n_e: 100 , l_r: 0.6 ,s 0.7 ,m_d 5 / 0.85 ,  1.0 ,  0.5714285714285714 ,  0.8125 ,  1.0
n_e: 100 , l_r: 0.8 ,s 0.7 ,m_d 9 / 0.85 ,  0.9230769230769231 ,  0.7142857142857143 ,  0.8571428571428571 ,  0.8333333333333334
n_e: 100 , l_r: 0.8 ,s 0.8 ,m_d 3 / 0.85 ,  1.0 ,  0.5714285714285714 ,  0.8125 ,  1.0
---

### 2-3. 임상정보 + 종양내외부 통합

In [14]:
# LDA

solver = ['lsqr', 'eigen','svd']
shrinkage = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for i in range(20,81,20):
    print("------------- reliefF ",i,"-------------------------------------")
    for a in solver:
        for b in shrinkage:
            train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:i]
            test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:i]

            total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
            total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

            X_train=total1.drop('two_year_survival',axis=1).values
            X_test=total2.drop('two_year_survival',axis=1).values
            y_train=total1['two_year_survival'].values
            y_test=total2['two_year_survival'].values
    
            if a=='svd': # solver='svd'면 shrinkage 파라미더 지정 불가 이므로 
                lda=LinearDiscriminantAnalysis(solver=a)
            else:
                lda=LinearDiscriminantAnalysis(solver=a, shrinkage=b)
            clf = make_pipeline(StandardScaler(), lda)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            
            # 성능평가
            accuracy = metrics.accuracy_score(y_test, y_pred)
            sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
            specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
            ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
            npv = metrics.precision_score(y_test, y_pred, pos_label=0)
            if accuracy>=0.7:
                print("solv:",a,",shrink:",b,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("------------- reliefF102-------------------------------------")
for a in solver:
    for b in shrinkage:
        train=patient_features_combine_train.drop('recurrence',axis=1)
        test=patient_features_combine_test.drop('recurrence',axis=1)

        total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
        total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

        X_train=total1.drop('two_year_survival',axis=1).values
        X_test=total2.drop('two_year_survival',axis=1).values
        y_train=total1['two_year_survival'].values
        y_test=total2['two_year_survival'].values

        if a=='svd': # solver='svd'면 shrinkage 파라미더 지정 불가 이므로 
            lda=LinearDiscriminantAnalysis(solver=a)
        else:
            lda=LinearDiscriminantAnalysis(solver=a, shrinkage=b)
        clf = make_pipeline(StandardScaler(),lda)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        # 성능평가
        accuracy = metrics.accuracy_score(y_test, y_pred)
        sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
        specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
        ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
        npv = metrics.precision_score(y_test, y_pred, pos_label=0)
        if accuracy>=0.7:
            print("solv:",a,",shrink:",b,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

------------- reliefF  20 -------------------------------------
------------- reliefF  40 -------------------------------------
------------- reliefF  60 -------------------------------------
solv: lsqr ,shrink: 0.6 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
solv: lsqr ,shrink: 0.7 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
solv: eigen ,shrink: 0.6 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
solv: eigen ,shrink: 0.7 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
------------- reliefF  80 -------------------------------------
solv: lsqr ,shrink: 0.6 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
solv: lsqr ,shrink: 0.7 / 0.7 ,  0.7692307692307693 ,  0.5714285714285714 ,  0.7692307692307693 ,  0.5714285714285714
solv: eigen ,shrink: 0.6 / 0.7 ,  

In [15]:
# SVM

C_set = [0.1,1, 10, 50, 100]
kernel_set = ["rbf"]
gamma_set = [0.001, 0.01, 0.1,0.5, 1]

for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in C_set:
        for b in kernel_set:
            for c in gamma_set:
                train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:i]
                test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:i]

                total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
                total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

                X_train=total1.drop('two_year_survival',axis=1).values
                X_test=total2.drop('two_year_survival',axis=1).values
                y_train=total1['two_year_survival'].values
                y_test=total2['two_year_survival'].values

                clf = make_pipeline(StandardScaler(), SVC(C=a,kernel=b,gamma=c,degree=d,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                # 성능평가
                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>0.7:
                    print("c:",a,",k:",b,",g:",c,", d:",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("------------- reliefF102-------------------------------------")
for a in C_set:
    for b in kernel_set:
        for c in gamma_set:
            train=patient_features_combine_train.drop('recurrence',axis=1)
            test=patient_features_combine_test.drop('recurrence',axis=1)

            total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
            total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

            X_train=total1.drop('two_year_survival',axis=1).values
            X_test=total2.drop('two_year_survival',axis=1).values
            y_train=total1['two_year_survival'].values
            y_test=total2['two_year_survival'].values

            clf = make_pipeline(StandardScaler(), SVC(C=a,kernel=b,gamma=c,degree=d,random_state=42))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            # 성능평가
            accuracy = metrics.accuracy_score(y_test, y_pred)
            sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
            specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
            ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
            npv = metrics.precision_score(y_test, y_pred, pos_label=0)
            if accuracy>0.7:
                print("c:",a,",k:",b,",g:",c,", d:",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

-----------------------reliefF 20 ---------------------
-----------------------reliefF 40 ---------------------
c: 50 ,k: rbf ,g: 0.001 , d: 9 / 0.75 ,  0.9230769230769231 ,  0.42857142857142855 ,  0.75 ,  0.75
-----------------------reliefF 60 ---------------------
c: 10 ,k: rbf ,g: 0.001 , d: 9 / 0.8 ,  1.0 ,  0.42857142857142855 ,  0.7647058823529411 ,  1.0
-----------------------reliefF 80 ---------------------
c: 10 ,k: rbf ,g: 0.001 , d: 9 / 0.75 ,  1.0 ,  0.2857142857142857 ,  0.7222222222222222 ,  1.0
------------- reliefF102-------------------------------------


In [16]:
# RF

n_estimators = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200]
max_depth = [None,2,4,6,8,10,12,14,16]
max_features_set = ["sqrt", "log2", "auto", None]

for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in n_estimators:
        for b in max_depth:
            for c in max_features_set:
                train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:i]
                test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:i]

                total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
                total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

                X_train=total1.drop('two_year_survival',axis=1).values
                X_test=total2.drop('two_year_survival',axis=1).values
                y_train=total1['two_year_survival'].values
                y_test=total2['two_year_survival'].values

                clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=a,max_depth=b,max_features=c,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                # 성능평가
                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>=0.7:
                    print("n_e:",a,",m_d:",b,",m_f",c,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("------------- reliefF102-------------------------------------")
for a in n_estimators:
    for b in max_depth:
        for c in max_features_set:
            train=patient_features_combine_train.drop('recurrence',axis=1)
            test=patient_features_combine_test.drop('recurrence',axis=1)

            total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
            total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

            X_train=total1.drop('two_year_survival',axis=1).values
            X_test=total2.drop('two_year_survival',axis=1).values
            y_train=total1['two_year_survival'].values
            y_test=total2['two_year_survival'].values

            clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=a,max_depth=b,max_features=c,random_state=42))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            # 성능평가
            accuracy = metrics.accuracy_score(y_test, y_pred)
            sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
            specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
            ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
            npv = metrics.precision_score(y_test, y_pred, pos_label=0)
            if accuracy>=0.7:
                print("n_e:",a,",m_d:",b,",m_f",c,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)   

-----------------------reliefF 20 ---------------------
n_e: 20 ,m_d: 2 ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 2 ,m_f log2 / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 2 ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 4 ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 4 ,m_f log2 / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 20 ,m_d: 4 ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 40 ,m_d: 2 ,m_f sqrt / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 40 ,m_d: 2 ,m_f log2 / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.7333333333333333 ,  0.6
n_e: 40 ,m_d: 2 ,m_f auto / 0.7 ,  0.8461538461538461 ,  0.42857142857142855 ,  0.73333333333333

n_e: 80 ,m_d: None ,m_f sqrt / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: None ,m_f auto / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 4 ,m_f sqrt / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 4 ,m_f auto / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 6 ,m_f sqrt / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 6 ,m_f auto / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 8 ,m_f sqrt / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 8 ,m_f auto / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 12 ,m_f sqrt / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 12 ,m_f auto / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.6842105263157895 ,  1.0
n_e: 80 ,m_d: 16 ,m_f sqrt / 0.7 ,  1.0 ,  0.14285714285714285 ,  0.68421052631578

KeyboardInterrupt: 

In [ ]:
# GB

n_estimators = [1,10,50, 100]
learning_rate = [0.1,0.3,0.5,0.6, 0.7,0.8,0.9]
subsample = [0.6,0.7,0.8,0.9,1.0]
max_depth = range(3,10,2)
    
for i in range(20,81,20):
    print("-----------------------reliefF",i,"---------------------")
    for a in n_estimators:
        for b in learning_rate:
            for c in subsample:
                for d in max_depth:
                    train=patient_features_combine_train.drop('recurrence',axis=1).iloc[:,:i]
                    test=patient_features_combine_test.drop('recurrence',axis=1).iloc[:,:i]

                    total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
                    total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

                    X_train=total1.drop('two_year_survival',axis=1).values
                    X_test=total2.drop('two_year_survival',axis=1).values
                    y_train=total1['two_year_survival'].values
                    y_test=total2['two_year_survival'].values

                    clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=a,learning_rate=b, subsample=c,max_depth=d,random_state=42))
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    # 성능평가
                    accuracy = metrics.accuracy_score(y_test, y_pred)
                    sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                    specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                    ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                    npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                    if accuracy>0.7:
                        print("n_e:",a,", l_r:",b,",s",c,",m_d",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)

print("------------- reliefF102-------------------------------------")
for a in n_estimators:
    for b in learning_rate:
        for c in subsample:
            for d in max_depth:
                train=patient_features_combine_train.drop('recurrence',axis=1)
                test=patient_features_combine_test.drop('recurrence',axis=1)

                total1=pd.merge(train,clinical_info_case2_train,on='PatientID')
                total2=pd.merge(test,clinical_info_case2_test,on='PatientID')

                X_train=total1.drop('two_year_survival',axis=1).values
                X_test=total2.drop('two_year_survival',axis=1).values
                y_train=total1['two_year_survival'].values
                y_test=total2['two_year_survival'].values

                clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=a,learning_rate=b, subsample=c,max_depth=d,random_state=42))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                # 성능평가
                accuracy = metrics.accuracy_score(y_test, y_pred)
                sensitivity = metrics.recall_score(y_test, y_pred, pos_label=1)
                specificity = metrics.recall_score(y_test, y_pred, pos_label=0)
                ppv = metrics.precision_score(y_test, y_pred, pos_label=1)
                npv = metrics.precision_score(y_test, y_pred, pos_label=0)
                if accuracy>0.7:
                    print("n_e:",a,", l_r:",b,",s",c,",m_d",d,"/",accuracy,', ',sensitivity,', ',specificity,', ',ppv,', ',npv)